In [3]:
# !pip install trl transformers accelerate peft datasets bitsandbytes einops pandas -U

In [4]:
from transformers import TextStreamer
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

In [5]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True)  # Set trust_remote_code=True
tokenizer.pad_token = tokenizer.eos_token  # Setting pad_token same as eos_token
tokenizer.padding_side = 'right'  # Setting padding_side to right
tokenizer.add_eos_token = True  # Setting add_eos_token to True
tokenizer.add_bos_token, tokenizer.add_eos_token
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [11]:
# unzip dataset
!unzip -q chat_dataset.zip

In [15]:
from datasets import load_from_disk
dataset = load_from_disk("chat_dataset")

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['chat', 'formatted_chat'],
        num_rows: 113
    })
    test: Dataset({
        features: ['chat', 'formatted_chat'],
        num_rows: 20
    })
})

In [19]:
# from datasets import Dataset
# dataset = Dataset.from_dict({"chat": chat})
# dataset = dataset.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chat"], tokenize=False, add_generation_prompt=False)})
# print(dataset['formatted_chat'][0])

In [20]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

In [21]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [22]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, peft_config)

In [24]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='mistralai/Mistral-7B-Instruct-v0.2', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=32, target_modules={'v_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [26]:
output_dir = "./output"
per_device_train_batch_size = 16 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 4  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = 320        # training will happen for 320 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    # bf16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to = "none"
)

In [28]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset['train'],
    eval_dataset =dataset['test'],
    peft_config=peft_config,
    dataset_text_field="formatted_chat",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
# upcasting the layer norms in torch.bfloat16 for more stable training
# for name, module in trainer.model.named_modules():
#     if "norm" in name:
#         module = module.to(torch.bfloat16)

In [29]:
# peft_model.config.use_cache = False
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacty of 15.77 GiB of which 576.38 MiB is free. Process 48433 has 15.21 GiB memory in use. Of the allocated memory 12.70 GiB is allocated by PyTorch, and 2.14 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
!pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import pandas as pd

In [ ]:
log_history = pd.DataFrame(trainer.state.log_history)

In [ ]:
log_history.shape

(49, 10)

In [ ]:
log_history.head()

,loss,grad_norm,learning_rate,epoch,step,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,3.1196,1.972712,0.000133,0.18,10,NaN,NaN,NaN,NaN,NaN
1,2.5625,0.446679,0.000200,0.36,20,NaN,NaN,NaN,NaN,NaN
2,2.2653,0.247179,0.000199,0.54,30,NaN,NaN,NaN,NaN,NaN
3,2.2436,0.243275,0.000199,0.71,40,NaN,NaN,NaN,NaN,NaN
4,2.1490,0.332283,0.000197,0.89,50,NaN,NaN,NaN,NaN,NaN


In [ ]:
log_history.sort_values(by = ['loss']).head()

,loss,grad_norm,learning_rate,epoch,step,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
41,1.2120,0.607608,8.104219e-06,7.50,420,NaN,NaN,NaN,NaN,NaN
45,1.2355,1.398129,9.115132e-07,8.21,460,NaN,NaN,NaN,NaN,NaN
44,1.2370,0.604082,2.047006e-06,8.04,450,NaN,NaN,NaN,NaN,NaN
38,1.2450,0.895122,1.792366e-05,6.96,390,NaN,NaN,NaN,NaN,NaN
34,1.2490,0.681968,3.615355e-05,6.25,350,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Loading original model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Loading PEFT model
PEFT_MODEL = f'pretrained'

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
merged_model = peft_model.merge_and_unload()

In [ ]:
### Save merged model

In [ ]:
# model.load_adapter("mental_health_model")
model.save_pretrained("./models", safe_serialization=True)

In [ ]:
peft_tokenizer.save_pretrained("./merged_models")

('./merged_models/tokenizer_config.json',
 './merged_models/special_tokens_map.json',
 './merged_models/tokenizer.model',
 './merged_models/added_tokens.json',
 './merged_models/tokenizer.json')

In [ ]:
from transformers import pipeline

def get_query(summary, query):
    return (
      "<s>You are an assistant for mental health."
      "\nBelow is a summary of the conversation and a new instruction."
      "\nWrite a response that is short, concise, and conversational. Remember, people prefer brief sentences."
      "\nPlease first identify the problem and state that, show empathy."
      "\nAlso, finish with a question that you'd like to ask."
      "Summary of conversation: {}"
        "\n[INST]{}[/INST]\n</s>\n\n".format(summary, query)
    )

# Generate a response using the updated eval_prompt
def get_answer(context, query):
    eval_promt = get_query(context, query)
    encoding = peft_tokenizer(eval_promt, return_tensors="pt")
    text_streamer = TextStreamer(peft_tokenizer)
    _ = merged_model.generate(**encoding, streamer = text_streamer, max_new_tokens = 512)

In [ ]:
get_answer("", "How can I prevent anxiety and depression?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health.
Below is a summary of the conversation and a new instruction.
Write a response that is short, concise, and conversational. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also, finish with a question that you'd like to ask.Summary of conversation: 
[INST]How can I prevent anxiety and depression?[/INST]
</s> 

Anxiety and depression are common feelings that everyone experiences at some point in their lives.  Preventing anxiety and depression is not always possible, but there are ways to manage these feelings.  

First, identify the problem.  What is causing you to feel anxious or depressed?  Is it a specific situation or person?  

Show empathy.  Say something like, "I understand that you are feeling anxious/depressed.  That's a difficult feeling to deal with."  

Finally, ask a question that will help you to understand more about the situation.  Ask, "What is causing you to feel anxi

In [ ]:
# get_answer("", "I am feeling depressed about my career progress, i feel like I am not growing")

In [ ]:
get_answer("", "Hello I am Mira. I am recent mom, why i feel blue now a days? could you please help?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health.
Below is a summary of the conversation and a new instruction.
Write a response that is short, concise, and conversational. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also, finish with a question that you'd like to ask.Summary of conversation: 
[INST]Hello I am Mira. I am recent mom, why i feel blue now a days? could you please help?[/INST]
</s> 

Hello Mira,

I'm so glad you reached out. You're a superhero for recognizing that you're feeling blue and wanting to address it. Postpartum depression is a real thing and it's important to take care of yourself.

You've just had a baby, so your body is going through a lot of changes. Hormones are all over the place and it can be overwhelming. You're also adjusting to a new role as a parent and possibly dealing with a lot of outside pressure.

You're not alone. Postpartum depression affects many new moms and dads. It's important to remem

In [ ]:
get_answer("Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health. Below is an instruction that describes a patient's problem.
Write a response that is concise and to the point. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also finish with a question that you like to ask
[INST]Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?[/INST]
</s> 

Hi Mira, I'm here to help. I understand that you're experiencing depression and you find that coitus is helpful. It's important to note that while sexual activity can have positive effects on mood, it's not a cure for depression. If you're feeling depressed, it might be helpful to speak with a mental health professional. They can provide you with effective treatments and strategies to manage your depression. Would you be open to exploring other options for improving your mood and overall mental health? For instance, regular exercise, a healthy diet, 

In [ ]:
context = """
[INST]Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?[/INST]
</s>

Hi Mira, I'm here to help. I understand that you're experiencing depression and you find that coitus is helpful. It's important to note that while sexual activity can have positive effects on mood, it's not a cure for depression. If you're feeling depressed, it might be helpful to speak with a mental health professional. They can provide you with effective treatments and strategies to manage your depression. Would you be open to exploring other options for improving your mood and overall mental health? For instance, regular exercise, a healthy diet, adequate sleep, and engaging in activities you enjoy can all contribute to better mental health. Do you have any specific interests or hobbies that you've been putting off due to your depression? I'd be happy to help you explore ways to incorporate these activities back into your life.

Additionally, have you considered seeking professional help from a therapist or counselor? They can provide you with evidence-based treatments and support to help you manage your depression. Would you be open to giving this a try?</s>
"""

In [ ]:
get_answer(context, "I understand, but I feel better in coitus")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health.
Below is an instruction that describes a patient's problem. please use previous conversation as context to answer
Write a response that is concise and to the point. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also finish with a question that you like to askPrevious conversation context: 
[INST]Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?[/INST]
</s>  

Hi Mira, I'm here to help. I understand that you're experiencing depression and you find that coitus is helpful. It's important to note that while sexual activity can have positive effects on mood, it's not a cure for depression. If you're feeling depressed, it might be helpful to speak with a mental health professional. They can provide you with effective treatments and strategies to manage your depression. Would you be open to exploring other options for improving